# Ciencia de Datos - TP6

## Integrantes

- Ambroa, Nicolás - 229/13 - ambroanicolas@hotmail.com
- Gaustein, Diego - 586/09 - diego@gaustein.com.ar

In [1]:
from collections import Counter
from functools import wraps
from itertools import chain
from string import punctuation
import os
import pickle
from pprint import pprint
from lxml import html
from nltk import sent_tokenize, word_tokenize, pos_tag, WordNetLemmatizer
from nltk.corpus import wordnet, stopwords
from sklearn.metrics import mutual_info_score

lemmatizer = WordNetLemmatizer()
stopwords = frozenset(stopwords.words('english'))

def load_or_call(func):
    """ Decorador auxiliar para cachear resultados en un .pickle. """
    @wraps(func)
    def wrapper(*args, **kwargs):
        filename = '{}-{}-{}.pickle'.format(func.__name__, str(args), str(kwargs))
        if os.path.exists(filename):
            with open(filename, 'rb') as f:
                return pickle.load(f)
        else:
            res = func(*args, **kwargs)
            with open(filename, 'wb') as f:
                pickle.dump(res, f)
            return res

    return wrapper

### 1.1. Levantar el corpus AP, separando cada noticia como un elemento distinto en un diccionario `(<DOCNO>: <TEXT>)`.

In [2]:
parsed = html.parse('ap/ap.txt')
documents = {}
for document in parsed.iter('doc'):
    docno, article = document.getchildren()
    documents[docno.text.strip()] = article.text.strip()

print('Cargados', len(documents), 'articulos')


Cargados 2250 articulos


### 1.2. Calcular el tamaño del vocabulario.

In [6]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    elif treebank_tag.startswith('S'):
        return wordnet.ADJ_SAT
    else:
        return wordnet.NOUN  # El default es NOUN

def get_words_for_document(document):
    for sentence in sent_tokenize(document):
        tagged_sentence = pos_tag(sentence.split())
        for word, pos in tagged_sentence:
            # Lematizar
            word = lemmatizer.lemmatize(word, pos=get_wordnet_pos(pos))
            # Strip punctuation
            word = word.strip(punctuation)
            # Lowercase
            word = word.lower()
            
            # Skip stopwords and punctuation
            if len(word) > 1 and word not in stopwords:
                yield word

@load_or_call
def get_word_count():
    c = Counter()
    for document in documents.values():
        c.update(get_words_for_document(document))
    return c
        
c = get_word_count()
cantidad_de_palabras = sum(c.values())
print(cantidad_de_palabras, 'palabras,', len(c), 'palabras distintas.')

Counter({'say': 9807, 'said': 4388, 'would': 2557, 'year': 2266, 'new': 2026, 'percent': 1942, 'one': 1764, 'people': 1665, 'also': 1621, 'million': 1617, 'two': 1570, 'u.s': 1532, 'president': 1497, 'government': 1463, 'last': 1458, 'make': 1455, 'take': 1313, 'official': 1296, 'go': 1295, 'state': 1258, 'report': 1229, 'time': 1098, 'first': 1091, 'police': 1087, 'could': 1051, 'include': 1029, 'get': 1011, 'soviet': 999, 'united': 994, 'tell': 990, 'day': 960, 'bush': 953, 'use': 923, 'billion': 886, 'call': 875, 'week': 871, 'three': 871, 'work': 858, 'since': 852, 'give': 847, 'states': 842, 'today': 841, 'come': 834, 'national': 827, 'company': 803, 'american': 796, 'plan': 796, 'house': 792, 'force': 766, 'group': 763, 'court': 751, 'thursday': 743, 'month': 732, 'federal': 730, 'high': 715, 'party': 714, 'city': 710, 'may': 700, 'tuesday': 696, 'market': 682, 'many': 679, 'want': 673, 'news': 672, 'wednesday': 670, 'think': 670, 'member': 668, 'country': 668, 'price': 667, 'mon

### 1.3. Para las 500 palabras con más apariciones, calcular el par más asociado según la medida presentada.

In [4]:
"""
Para cada palabra, aproximamos p(palabra) con su frecuencia relativa según el counter.
Tomamos las 500 con más apariciones y recorremos el texto. Cada vez que encontremos una de estas miramos en una
ventana (n=8) cuando aparece alguna de las otras, y le sumamos uno.
Esta es nuestra estimación de p(palabra1, palabra2). Finalmente calculamos la información mutua e imprimimos los
pares más asociados.
"""

# Función que calcula la información mutua.
def calc_MI(x, y):
    c_xy = np.histogram2d(x, y, bins_entropia)[0]
    mi = mutual_info_score(None, None, contingency=c_xy)
    return mi

def mirar_ventana_y_buscar(palabra_a_asociar, palabras, palabras_a_buscar, indice_inicial, longitud_de_ventana, asociaciones):
    # Arreglo los índices para no pasarme en caso de una ventana muy cercana a los extremos de la lista.
    if indice_inicial - longitud_de_ventana < 0:
        indice_inicial = 0
    else:
        indice_inicial -= longitud_de_ventana
    if indice_inicial + longitud_de_ventana > len(palabras):
        indice_final = len(palabras)
    else:
        indice_final = indice_inicial + longitud_de_ventana
       
    # Recorro los indices, buscando palabras a buscar que no sean la palabra a asociar.
    for indice in range(indice_inicial, indice_final):
        if palabras[indice] in palabras_a_buscar and palabras[indice] != palabra_a_asociar:
            subindice_dict = str(palabra_a_asociar) + " " +str(palabras[indice])
            # Si la encontre, aumento en uno la asociacion entre ambas palabras.
            try:
                asociaciones[subindice_dict] += 1
            except IndexError:
                asociaciones[subindice_dict] = 1

@load_or_call
def contar_asociaciones_de_palabras():
    contador_asociaciones_de_palabras = Counter()
    # Recorremos el texto.
    for document in documents.values():
        palabras = document.split()
        for indice, palabra in enumerate(palabras):
            #Cada vez que encontremos una de ellas, miramos en una ventana de n=8.
            if palabra in palabras_con_mas_apariciones:
                mirar_ventana_y_buscar(palabra, palabras, palabras_con_mas_apariciones, indice, 8, contador_asociaciones_de_palabras)
    return contador_asociaciones_de_palabras

# Tomamos las 500 palabras con más apariciones.
palabras_y_apariciones = c.most_common(500)
palabras_con_mas_apariciones = [tupla[0] for tupla in palabras_y_apariciones]

asociaciones_de_palabras = contar_asociaciones_de_palabras()
# Recorremos los pares y calculamos información mutua para cada uno.
for palabras, asociaciones in asociaciones_de_palabras.items():
    palabras = palabras.split(" ")
    palabra_x = palabras[0]
    palabra_y = palabras[1]

# Finalmente calculamos inf mutua y imprimos el par + asoc según esta medida.
            

['much', 'television']
['need', 'percent']
['step', 'say']
['chief', 'even']
['water', 'three']
['trading', 'oil']
['week', 'financial']
['news', 'statement']
['international', 'many']
['student', 'strong']
['get', 'car']
['great', 'long']
['meeting', 'continue']
['long', 'show']
['leader', 'market']
['decision', 'conference']
['water', 'say']
['charge', 'second']
['start', 'aid']
['television', 'like']
['even', 'report']
['move', 'general']
['get', 'order']
['president', 'approve']
["don't", 'free']
['reduce', 'effort']
['turn', 'official']
['worker', 'economy']
['whether', 'give']
['15', 'six']
['spokesman', 'approve']
['years', 'communist']
['bill', 'support']
['decision', 'know']
['near', 'years']
['two', '1988']
['new', 'campaign']
['several', 'chairman']
['whose', 'states']
['western', 'possible']
['12', 'eight']
['say', 'part']
['north', 'along']
['involve', 'fight']
['last', 'car']
['take', 'price']
['two', 'see']
['interest', 'well']
['system', 'less']
['rate', 'call']
['north

['break', 'still']
['report', 'western']
['change', 'three']
['interest', 'support']
['position', 'family']
['government', '50']
['president', 'thing']
['without', 'also']
['release', 'result']
['25', 'release']
['charge', 'officer']
['still', 'show']
['show', 'long']
['political', '1988']
['today', 'board']
['since', 'total']
['100', 'fight']
['peace', 'man']
['set', 'said']
['make', 'head']
['others', 'security']
['drug', 'military']
['large', 'plant']
['away', 'fire']
['states', 'economic']
['bank', 'buy']
['fire', 'city']
['president', 'demand']
['budget', 'send']
['town', 'school']
['last', 'another']
['provide', "don't"]
['contract', 'federal']
['central', 'new']
['stock', '1986']
['official', 'health']
['vice', 'international']
['week', 'election']
['fall', 'side']
['exchange', 'would']
['play', 'administration']
['radio', 'director']
['hope', 'one']
['although', 'would']
['four', 'today']
['capital', 'district']
['people', 'force']
["nation's", 'open']
['next', 'said']
['need',

['line', 'begin']
['appeal', 'death']
['show', 'continue']
['power', 'seven']
['away', 'hold']
['month', 'break']
['enough', 'member']
["don't", 'ask']
['appeal', 'university']
['new', 'require']
['member', 'said']
['general', 'bring']
['state', 'big']
['even', 'system']
['much', 'system']
['market', 'presidential']
['face', 'peace']
['southern', 'last']
['food', 'among']
['statement', 'death']
['trade', 'good']
['include', 'among']
['three', 'believe']
['provide', 'water']
['half', 'across']
['first', 'human']
['world', 'total']
['time', 'flight']
['free', 'major']
['years', 'name']
['national', 'good']
['deal', 'night']
['today', 'trade']
['official', 'report']
['seven', '30']
['rate', 'every']
['information', 'reporter']
['sell', 'system']
['already', 'percent']
['around', 'far']
['human', 'good']
['chief', 'prime']
['kill', 'try']
['case', 'like']
['three', 'possible']
['work', 'high']
['political', 'trading']
['possible', 'board']
['rate', 'one']
["that's", 'police']
['outside', '

['serve', 'total']
['rule', 'private']
['board', 'time']
['billion', 'close']
['charge', 'place']
['general', 'one']
['protest', 'capital']
['aid', 'food']
['second', 'talk']
['aid', 'pay']
['total', 'near']
['board', 'company']
['several', 'visit']
['still', '1986']
['level', 'international']
['claim', 'pay']
['company', 'administration']
['without', 'country']
["don't", 'first']
['need', 'start']
['food', 'stand']
['take', 'private']
['federal', 'chief']
['many', 'never']
['black', 'open']
['one', 'fight']
['policy', 'drug']
['record', 'day']
['set', 'trade']
['also', 'high']
['agreement', 'call']
['whose', 'number']
['year', 'party']
['also', 'tax']
['policy', 'spokesman']
['100', 'carry']
['would', 'report']
['increase', 'second']
['rate', 'increase']
['town', 'show']
['policy', 'use']
['last', 'today']
['since', 'information']
['also', 'report']
['job', 'ago']
['official', 'free']
['30', 'public']
['must', 'government']
['medical', 'try']
['said', 'system']
['change', 'member']
['

['vote', 'country']
['part', '20']
['tax', 'long']
['30', 'committee']
['day', 'home']
['years', 'early']
['three', 'seven']
['time', "didn't"]
['program', 'leader']
['kill', 'much']
['fight', 'said']
['federal', 'still']
['ago', 'director']
['1988', 'term']
['see', 'interest']
['cut', 'half']
['head', 'many']
["don't", 'result']
['outside', 'death']
['drop', 'also']
['child', 'last']
['cut', '50']
['little', 'offer']
['late', 'aid']
['meeting', 'three']
['south', '30']
['last', 'operation']
['number', 'government']
['total', 'also']
['three', 'study']
['company', 'might']
['defense', 'without']
['television', 'city']
['less', 'show']
['central', 'visit']
['action', 'although']
['said', 'show']
['fell', 'plane']
['another', 'fell']
['meet', 'would']
['could', 'percent']
['foreign', 'past']
['oil', 'offer']
['power', 'campaign']
['raise', 'defense']
['sale', 'since']
['1986', 'control']
["nation's", 'government']
['pay', 'raise']
['town', 'south']
['could', 'condition']
['fire', 'condit

...

['hour', "it's"]
['index', 'company']
['take', 'pay']
['base', 'around']
['carry', 'agree']
['former', 'support']
['could', 'along']
['economy', 'news']
['vice', 'use']
['former', 'eight']
['public', 'food']
['press', 'school']
['result', 'even']
['president', 'last']
['near', 'council']
['board', 'put']
['member', 'still']
['15', 'since']
['still', 'number']
['sell', 'new']
['well', 'capital']
['already', 'go']
['question', "didn't"]
['100', 'leave']
['base', "didn't"]
['like', 'foreign']
['begin', 'work']
['make', 'money']
['even', 'great']
['years', 'general']
['house', 'police']
['remain', 'percent']
['cut', 'least']
['capital', 'new']
['change', "it's"]
['communist', 'last']
['union', '10']
['say', 'show']
['offer', 'far']
['would', 'nation']
['local', 'official']
['job', 'problem']
['dollar', 'vote']
['one', 'state']
['increase', 'whether']
['president', 'company']
['executive', 'ago']
['think', 'political']
['month', 'visit']
['court', 'press']
['50', 'nearly']
['president', 'on

['week', 'white']
['three', 'president']
['old', 'nearly']
['well', 'among']
['army', 'head']
['remain', 'service']
['billion', 'ask']
['20', 'five']
['one', 'stand']
['may', 'require']
['kill', 'eight']
['television', 'around']
['policy', 'government']
['family', 'early']
['years', 'job']
['look', 'government']
['become', 'use']
['small', 'man']
['two', 'help']
['already', 'bill']
['little', 'late']
['million', 'cut']
['50', '1988']
['court', 'next']
['support', 'secretary']
['30', 'make']
['life', 'take']
['administration', 'think']
['former', 'get']
['take', 'soldier']
['another', 'away']
['last', "don't"]
['fall', 'major']
['organization', 'men']
['foreign', 'opposition']
['later', 'bring']
['would', 'state']
['executive', 'agency']
['vice', 'bring']
['month', 'trial']
['nation', 'law']
['family', 'administration']
['capital', 'tax']
['sell', 'agree']
['water', 'study']
['25', 'allow']
['judge', 'trial']
["don't", 'back']
['break', 'might']
['human', 'think']
['control', 'war']
['m

['three', 'right']
['statement', 'least']
['hold', 'former']
['water', 'pay']
['spokesman', 'years']
['official', 'month']
['try', 'school']
['30', '15']
['go', 'time']
['department', 'others']
['first', 'eight']
['second', 'television']
['council', 'head']
['white', 'men']
['news', 'act']
['company', 'price']
['file', 'would']
['whether', 'family']
['two', 'contract']
['raise', 'want']
['state', 'lose']
['comment', 'industry']
['vice', 'play']
['measure', 'fight']
['many', 'economic']
['1986', 'give']
['could', 'member']
['said', 'vice']
['executive', 'court']
['five', '50']
['begin', 'many']
['end', 'first']
['today', 'would']
['military', 'come']
['first', 'half']
['since', 'meeting']
['case', 'state']
["didn't", 'woman']
['family', 'take']
['country', 'financial']
['money', 'drug']
['condition', 'men']
['million', "we're"]
['fight', 'even']
['among', 'since']
['talk', 'spokesman']
['die', 'first']
['possible', 'said']
['student', 'many']
['child', 'woman']
['although', 'average']
[

['win', 'military']
['information', 'last']
['new', 'program']
['home', 'make']
['near', 'tax']
['market', '12']
['vice', 'condition']
['law', 'comment']
['special', 'half']
['southern', 'area']
["don't", 'set']
['without', 'day']
['could', 'flight']
['see', 'make']
['part', 'people']
['10', 'country']
['effort', 'cut']
['two', 'cost']
['fight', 'agreement']
['since', 'week']
['men', 'fire']
['reach', "don't"]
['one', 'nearly']
['without', 'second']
['need', 'home']
['conference', 'without']
['whether', 'support']
['future', 'party']
['outside', 'go']
['outside', 'study']
['years', 'keep']
['pay', '30']
['first', 'interest']
['minister', 'health']
['southern', 'percent']
['many', 'across']
['report', 'former']
['could', 'bill']
['people', 'department']
['second', 'service']
['average', 'week']
['offer', 'begin']
['foreign', 'fall']
['southern', 'year']
['issue', 'could']
['news', "don't"]
['agreement', 'begin']
['cut', 'agency']
['chief', 'use']
['east', 'five']
['hospital', "don't"]
[

['near', 'live']
['price', 'must']
['nation', 'put']
['man', '10']
['states', 'eight']
['exchange', 'percent']
['comment', 'security']
['think', 'end']
['visit', 'official']
['support', 'central']
['along', 'report']
['result', 'much']
['think', 'run']
['also', '1988']
['rise', 'people']
['still', 'four']
['foreign', 'know']
['dollar', 'come']
['major', 'early']
['price', 'people']
['military', 'power']
['cut', 'several']
['decline', 'department']
['southern', 'low']
['long', 'step']
['would', 'possible']
['administration', 'since']
['raise', 'good']
['like', 'decision']
['six', 'call']
['decision', 'get']
['next', 'provide']
['concern', 'year']
['live', 'death']
['police', 'meet']
['department', '20']
['country', 'human']
['past', 'see']
['family', 'many']
['high', 'group']
['five', 'next']
['support', 'large']
['party', 'attorney']
['trial', 'government']
['business', 'major']
['officer', 'would']
['whether', 'union']
['court', 'case']
['pay', 'know']
['would', 'raise']
['government'